In [114]:
import copy, numpy as np
np.random.seed(0)

In [115]:
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

In [116]:
def sigmoid_derivative(activation):
    return activation*(1-activation)

In [117]:
def loss_derivative(ycap,y):
    return 2*(ycap-y)

In [118]:
np.array([range(3)])
np.shape(np.zeros((6,8)).T)

(8, 6)

In [119]:
int2binary = {}
binary_dim = 8

largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)],dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary[i] = np.reshape(binary[i],(1,8))

In [120]:
np.shape(binary)

(256, 8)

In [121]:
int2binary[1]

array([[0, 0, 0, 0, 0, 0, 0, 1]], dtype=uint8)

In [122]:
#hyperparameters
alpha=0.1
in_dim = 16
hidden_dim = 32
out_dim = 8

In [123]:
Wx = np.random.random((in_dim,hidden_dim))
bx = np.random.random((hidden_dim,1))
Wy = np.random.random((hidden_dim, out_dim))
by = np.random.random((out_dim,1))

In [130]:
# generate a simple addition problem (a + b = c)
a_int = np.random.randint(largest_number/2) # int version
a = int2binary[a_int] # binary encoding

b_int = np.random.randint(largest_number/2) # int version
b = int2binary[b_int] # binary encoding
ab = np.concatenate((a,b),axis=1)
# true answer
c_int = a_int + b_int
c = int2binary[c_int]
pred = np.zeros_like(c)
Zx = np.matmul(ab,Wx)+bx.T
Ax = 1/(1+np.exp(-Zx))
Zy = np.matmul(Ax,Wy)+by.T
Ay = 1/(1+np.exp(-Zy))
loss = (c - Ay)**2

#backpropagation

dAy = 2*(Ay-c)
dZy = dAy*Ay*(1-Ay)

#in doubt
dWy = np.matmul(Ax.T,dZy)
dBy = copy.deepcopy(dZy)
dAx = np.matmul(Wy,dZy)
# dAx = dZy*Wy

In [131]:
dWy

array([[-1.16863148e-16,  2.82378436e-06, -1.60055855e-15,
        -2.68684056e-15, -8.80271902e-12, -6.71728997e-14,
        -2.37733050e-14,  3.02284200e-07],
       [-1.15959896e-16,  2.80195893e-06, -1.58818761e-15,
        -2.66607359e-15, -8.73468155e-12, -6.66537108e-14,
        -2.35895578e-14,  2.99947803e-07],
       [-1.16284505e-16,  2.80980251e-06, -1.59263346e-15,
        -2.67353678e-15, -8.75913272e-12, -6.68402959e-14,
        -2.36555925e-14,  3.00787453e-07],
       [-1.16053226e-16,  2.80421408e-06, -1.58946586e-15,
        -2.66821938e-15, -8.74171164e-12, -6.67073569e-14,
        -2.36085438e-14,  3.00189215e-07],
       [-1.17945958e-16,  2.84994850e-06, -1.61538874e-15,
        -2.71173583e-15, -8.88428174e-12, -6.77952988e-14,
        -2.39935796e-14,  3.05085054e-07],
       [-1.16475694e-16,  2.81442224e-06, -1.59525198e-15,
        -2.67793248e-15, -8.77353404e-12, -6.69501913e-14,
        -2.36944858e-14,  3.01281992e-07],
       [-1.19082147e-16,  2.877402

In [38]:
for j in range(10000):
    a_int = np.random.randint(largest_number/2) # int version
    a = int2binary[a_int] # binary encoding

    b_int = np.random.randint(largest_number/2) # int version
    b = int2binary[b_int] # binary encoding
    ab = np.concatenate((a,b),axis=1)
    # true answer
    c_int = a_int + b_int
    c = int2binary[c_int]
    pred = np.zeros_like(c)
    Zx = np.matmul(ab,Wx)+bx.T
    Ax = 1/(1+np.exp(-Zx))
    Zy = np.matmul(Ax,Wy)+by.T
    Ay = 1/(1+np.exp(-Zy))
    
    loss = 
    


array([[0.74009753],
       [0.68051448],
       [0.62238443],
       [0.7105284 ],
       [0.20492369],
       [0.34169811],
       [0.67624248],
       [0.87923476],
       [0.54367805],
       [0.28269965],
       [0.03023526],
       [0.71033683],
       [0.0078841 ],
       [0.37267907],
       [0.53053721],
       [0.92211146]])